# Birdsongs - 03.- Obteniendo datos - Descargando Audios

A partir del dataset procede a realizar las descargas de los ficheros de audio de [**Xeno-Canto**](https://www.xeno-canto.org).

Con el dataset obtenido desde **Birdsongs_02_Obteniendo Datos_Seleccionando_especies**, descarga los audios en un directorio local, en formato mp3. Cada una de las grabaciones se descarga en un directorio con el nombre de la especie al que pertenece, quedando una estructura de directorios del tipo

![audiofiles](./resources/directorioaudios.png)


Los audios se encuentran almacenados en la web con la siguiente estructura de directorios:

* https://www.xeno-canto.org/
* ID de la grabación
* download

Ejemplo:
https://www.xeno-canto.org/416171/download

## 1.- Librerías

Cargamos las librerías utilizadas en el notebook


Para realizar la descarga de los ficheros desde la web vamos a utilizar 
[**wget**](https://www.gnu.org/software/wget/) que permite recuperar ficheros a través de internet

>GNU Wget is a free software package for retrieving files using HTTP, HTTPS, FTP and FTPS the most widely-used Internet protocols. It is a non-interactive commandline tool, so it may easily be called from scripts, cron jobs, terminals without X-Windows support, etc.

    pip install wget


In [ ]:
# importar librerías
import pandas as pd
import wget
import os


## 2.- Funciones

### Descarga ficheros

Va iterando sobre el dataset y va recuperando las grabaciones

In [ ]:
#----------------------------------------------------------------------------
# download_songs(df_path, download_url, local_path)
#  argumentos: 
#      df_path: fichero csv con el datset
#      download_url: url de descarga
#      local_path: directorio de destino donde se guardan los audios
#----------------------------------------------------------------------------
def download_songs(df_path, download_url, local_path):
    print(">>> downloading songs...")

    # cargar dataframe a partir de fichero csv
    df_birdsongs = pd.read_csv(df_path)
    
    # crear el directorio local de descarga si no existe
    if not os.path.isdir(local_path):
        os.mkdir(local_path)    

    # si el dataframe contiene grabaciones, procede a descargarlas de la web
    if df_birdsongs.shape[0] > 0:
        # iterar sobre el dataframe
        for index, row in df_birdsongs.iterrows():
            # Identificador de la grabación y nombre de la especie
            ID = row["ID"]
            name = row["Name"]

            print(ID + '-' + name, end='\r', flush=True)

            # Crea el directorio de la especie si no existe
            folder = os.path.join(local_path, name)
            
            if not os.path.exists(folder):
                os.mkdir(folder)

            # Descargar el fichero si no existe
            file = ID + '.mp3'
            filepath = os.path.join(local_path, name, file)

            if not os.path.isfile(filepath):
                download_file = os.path.join(download_url, ID[2:], 'download')

                # descarga audio
                try:
                    wd = wget.download(download_file, filepath)

                # controla excepciones
                except Exception as inst:
                    print(file)            
                    print("Error al intentar la descarga: ", inst)
                    
                    pass                


## 3.- Descarga audios

Va iterando sobre el dataset, y para cada una de las grabaciones:

* Comprueba si el directorio para la especie existe y si no lo crea
* Comprueba si ya se ha descargado el audio correspondiente a la grabación (si existe no lo descarga)
* Descarga la grabación

En caso que el proceso se interrumpa, se podrá volver a lanzar tantas veces como sea necesario, y continuará descargando aquellos audios que estén pendientes.


In [ ]:
%%time

# fichero csv con el dataset
df_path = 'Birdsongs_My_Birdsongs_Europe_20190107211642.csv'

# URL de la página web donde se encuentran los audios
download_url = 'https://www.xeno-canto.org/'  

# directorio local de descarga
local_path = './audio/' 

# descarga audios
download_songs(df_path, download_url, local_path) 
